In [1]:
import scipy.io
import numpy as np
import glob
import cv2
from google.colab.patches import cv2_imshow
from functools import reduce 
import math

In [2]:
path = "/content/masks/*.*"
masks = []
for file in glob.glob(path):
   mask = scipy.io.loadmat(file)
   masks.append(list(mask['mask'].flatten()))
masks = np.array(reduce(lambda a, b: a+b, masks))

In [3]:
images = []
path = "/content/images/*.*"
for file in glob.glob(path):
   img =  cv2.imread(file)
   gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
   images.append(list(gray.flatten()))
images = np.array(reduce(lambda a, b: a+b, images))

In [4]:
mean = sum(np.multiply(images, masks))/sum(masks)
var = sum((x - mean)**2 * m for (x, m) in zip(images, masks)) / sum(masks)
print(mean, var) # for class 'face'

98.06235876967752 4816.7060079930225


In [5]:
non_masks = 1 - masks
mean = sum(np.multiply(images, non_masks))/sum(non_masks)
var = sum((x - mean)**2 * m for (x, m) in zip(images, non_masks)) / sum(non_masks)
print(mean, var) # for class 'non-face'

104.36191646128414 4820.033064337396


In [10]:
prior = {'face': 0.22194392442521707, 'non-face': 0.778056075574783}
mean = {'face': 98.06235876967752 , 'non-face': 104.36191646128414}
var = {'face': 4816.7060079930225, 'non-face': 4820.033064337396}

In [11]:
def classifier(test):

  predicted = np.zeros(len(test))

  for i in range(len(test)):

    probabilities = {'face': 0, 'non-face': 0}

    exponents = {'face': math.exp(-(math.pow(test[i] - mean['face'], 2)/(2 * var['face']))),
                 'non-face': math.exp(-(math.pow(test[i] - mean['non-face'], 2)/(2 * var['non-face'])))}

    probabilities['face'] = prior['face'] * (1 / (math.sqrt(2 * math.pi * var['face']))) * exponents['face']
    probabilities['non-face'] = prior['non-face'] * (1 / (math.sqrt(2 * math.pi * var['non-face']))) * exponents['non-face']

    bestLabel, bestProb = None, -1

    if probabilities['face'] >= probabilities['non-face']:
       predicted[i] = 1
    else:
      predicted[i] = 0

  return predicted

In [ ]:
test_image = cv2.cvtColor(cv2.imread("img_01.jpg"), cv2.COLOR_BGR2GRAY)
test = test_image.flatten()
print(test)
prediction = classifier(test)
print(prediction)
cv2_imshow(prediction.reshape(test_image.shape[0], test_image.shape[1]))